# Example 10-1. Character level language model - Dinosaur

공룡 이름 생성기 만들기 

- 기존의 공룡 이름 데이터를 사용하여 RNN 모형을 만들고 자동으로 이름 생성 
- Sequence-to-sequence 형태의 모형을 사용 

<img src="figures/dinosour.png" width="70%">
<img src="figures/dinosour2.png" width="70%">


In [4]:
import numpy as np
import random
from random import shuffle
from keras.preprocessing import sequence
from keras.utils import to_categorical

## 데이터 불러오기


- 이름 사이는 enter(`\n`)가 들어가 있음
- 이름의 시작을 tab(`\t`)으로 구분할 것이기 때문에 `\t`를 `\chars`에 추가

In [5]:
data = open('data/dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
chars.append('\t') 

<font color=blue>
TO DO:
    
- 데이터가 전체는 총 몇 개의 문자로 이루어져 있는가?
- Vocabulary는 몇 개의 문자로 이루어져 있는가?

In [6]:
# Your answer comes here
print(len(data),
len(chars))

19909 28


<font color=blue>
    
TO DO:

- char와 index를 연결하는 lookup table을 만드시오. (`char2index`, `index2char`)

In [7]:
# Your answer comes here
char2index = dict((c, i) for i, c in enumerate(sorted(chars)))
index2char = dict((i, c) for i, c in enumerate(sorted(chars)))

## Train data 만들기 

<font color=blue>
    
TO DO: 
    
- `examples`의 이름으로 각 줄을 읽어들임
- 공백을 없애고 소문자로 변환하여 이름 list 생성 

In [8]:
# -*- coding: utf-8 -*-
INPUT_FILE = "data/dinos.txt"

fin = open(INPUT_FILE, 'rb') # 바이너리 파일을 읽기 모드로 오픈

examples = []
i=0
for line in fin: # 파일을 한 줄씩 읽어들임
    line = line.strip().lower() # 공백을 제거하고 소문자로 변환
    line = line.decode("ascii") # 디코딩하여 char로 변환
    if len(line) == 0: # 빈 줄 삭제
        continue
    examples.append(line)
fin.close()

<font color=blue>
    
TO DO: 
    
- 이름의 최대 길이는 몇 인가? 즉, 가장 긴 이름은 몇 개의 문자로 이루어져 있는가?

In [9]:
# Your answer comes here
max_len = max([len(line) for line in examples])
max_len

26

- 생성할 가장 긴 이름을 `\n`, `\t` 포함 30개 문자로 이루어지도록 제한

In [10]:
max_len = 30
nb_chars = len(char2index)

<font color=blue>
    
TO DO: 
    
아래의 사항을 포함하여 문자로 이루어진 이름을 각 문자를 나타내는 one-hot vector의 시퀀스로 변환하여 sequence-to-sequence 모델의 input(X)와 output(Y)를 만드시오. 

- 이름의 시작에는 `\t`붙이기
- 이름의 끝에는 `\n` 붙이기
- 총 이름의 길이는 `max_len` (`max_len`보다 짧은 경우 뒤를 `\n`으로 채움) 
- Y는 X 입력값의 다음 문자를 지정 

- E.g. example[0]: aachenosaurus
      input: \t a a c h e n n o s a u r u s \n \n \n .... (30개 채울 때 까지)
      output: a a c h e n n o s a u r u s \n \n \n .... (30개 채울 때 까지)
        
        X[0]= array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
        ...
        
  

In [11]:
X = np.zeros((len(examples), max_len, nb_chars), dtype=int)
Y = np.zeros((len(examples), max_len, nb_chars), dtype=int)

for i, example in enumerate(examples):
    example_x = '\t'+example
    if max_len > len(example_x):
        example_x = example_x +'\n' * (max_len - len(example_x))
        example_y = example + '\n' * (max_len - len(example))
    for j, character in enumerate(example_x):
        X[i, j, char2index[character]] = 1.
    for j, character in enumerate(example_y):
        Y[i, j, char2index[character]] = 1.       

In [12]:
X.shape, Y.shape

((1536, 30, 28), (1536, 30, 28))

## 모델 만들기 

In [13]:
from keras.models import Model, Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras import optimizers
from keras import backend as K
import time 

In [18]:
# Your answer comes here
HIDDEN_SIZE = 64
BATCH_SIZE = 128

K.clear_session()
model = Sequential()
model.add(LSTM(HIDDEN_SIZE, return_sequences=True, input_shape=(max_len, nb_chars), activation='relu'))
model.add(LSTM(HIDDEN_SIZE, return_sequences=True, activation='relu'))
model.add(Dense(HIDDEN_SIZE, activation='relu'))
model.add(Dense(nb_chars, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer=optimizers.adam(lr=0.001))

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30, 64)            23808     
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 64)            33024     
_________________________________________________________________
dense_1 (Dense)              (None, 30, 64)            4160      
_________________________________________________________________
dense_2 (Dense)              (None, 30, 28)            1820      
Total params: 62,812
Trainable params: 62,812
Non-trainable params: 0
_________________________________________________________________


In [19]:
now = time.strftime("%c")
callbacks_list = [
    ModelCheckpoint(filepath='Dinosaur_gen.h5', monitor='val_loss', save_best_only=True),
    TensorBoard(log_dir='/home/work/logs/Dinosaur_generation/'+now),
    EarlyStopping(monitor='val_loss',patience=3)
]
model.fit(X, Y, batch_size=BATCH_SIZE, epochs=100, validation_split=0.2, callbacks=callbacks_list)


Train on 1228 samples, validate on 308 samples
Epoch 1/100
1228/1228 [==============================] - 2s 2ms/step - loss: 3.2720 - val_loss: 3.1611
Epoch 2/100
1228/1228 [==============================] - 1s 681us/step - loss: 2.7813 - val_loss: 1.6142
Epoch 3/100
1228/1228 [==============================] - 1s 669us/step - loss: 1.5848 - val_loss: 1.4614
Epoch 4/100
1228/1228 [==============================] - 1s 670us/step - loss: 1.4327 - val_loss: 1.4008
Epoch 5/100
1228/1228 [==============================] - 1s 669us/step - loss: 1.3873 - val_loss: 1.3061
Epoch 6/100
1228/1228 [==============================] - 1s 686us/step - loss: 1.3314 - val_loss: 1.2769
Epoch 7/100
1228/1228 [==============================] - 1s 666us/step - loss: 1.3022 - val_loss: 1.2664
Epoch 8/100
1228/1228 [==============================] - 1s 685us/step - loss: 1.2870 - val_loss: 1.2466
Epoch 9/100
1228/1228 [==============================] - 1s 668us/step - loss: 1.2748 - val_loss: 1.2403
Epoch 10/1

## 공룡 이름 생성하기 

In [21]:
for i in range(100):
    stop = False
    ch = "\t"
    counter = 1
    target_seq = np.zeros((1,max_len, nb_chars))
    target_seq[0, 0, char2index[ch]] = 1
    while stop == False and counter < max_len:
        probs = model.predict_proba(target_seq, verbose=0)[:, counter-1, :]
        c = np.random.choice(sorted(list(chars)), replace=False, 
                             p=probs.reshape(nb_chars))
        if c=="\n":
            stop=True
        else:
            ch=ch+c
            target_seq[0, counter, char2index[c]] = 1
            counter = counter + 1
    print(ch) 

	jajhtocronsaurus
	bonlausaurus
	eparorarator
	wpniianosaurus
	speicaropsaurus
	ppuacletocecpsulus
	hhocrlorkax
	danongonnosaurusysxurasaesaor
	logmotan
	eadospenlus
	hnfnatontoplitas
	rykiexathus
	coceratyrax
	thaqbaunn
	dhrortosurus
	wrbosurus
	aihaocoralosausaurasrus
	jgothansaurus
	idapirodesaurus
	iunatprosaurus
	corosaurus
	sanonanisaurus
	jolkcosxathosaulus
	degnaisurus
	gnnsaiepesaurus
	outnposaurus
	eproi
	clelaynus
	komolinasorus
	onansgraphis
	anjyasaurus
	ndnirosaurus
	eirchhanamus
	gceasinis
	aeugderamos
	anemeriosaurus
	cmanopnnysaurus
	aiwnosaurus
	bistesaurus
	mudrimiodosaurus
	ageharoshurus
	ianinamas
	banctonusaurus
	radrolonatosachus
	manokanatan
	deeyerangosaurus
	eanlyricoraus
	dprasaurivosaurus
	iraesaurus
	idicanax
	dcerolaceroppx
	duontemtops
	gagisasurus
	euserapenisorurisaurus
	aamanamanx
	cblionon
	cprlaporan
	ctianhinysaurus
	pegeosaurus
	lancodrahantor
	bacplomosorus
	protanhigvoras
	lharesitesaurus
	boritopsgus
	eelansionoras
	muuatrosaurus
	drzantosaurusd